<a href="https://colab.research.google.com/github/nghiavuPrac/nerfstudio-testing/blob/main/Copy_of_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
    <picture>
    <source media="(prefers-color-scheme: dark)" srcset="https://docs.nerf.studio/en/latest/_images/logo-dark.png">
    <source media="(prefers-color-scheme: light)" srcset="https://docs.nerf.studio/en/latest/_images/logo.png">
    <img alt="nerfstudio" src="https://docs.nerf.studio/en/latest/_images/logo.png" width="400">
    </picture>
</p>


# Nerfstudio: A collaboration friendly studio for NeRFs


![GitHub stars](https://img.shields.io/github/stars/nerfstudio-project/nerfstudio?color=gold&style=social)

This colab shows how to train and view NeRFs from Nerfstudio both on pre-made datasets or from your own videos/images.

\\

Credit to [NeX](https://nex-mpi.github.io/) for Google Colab format.

## Frequently Asked Questions

*  **Downloading custom data is stalling (no output):**
    * This is a bug in Colab. The data is processing, but may take a while to complete. You will know processing completed if `data/nerfstudio/custom_data/transforms.json` exists. Terminating the cell early will result in not being able to train.
*  **Processing custom data is taking a long time:**
    * The time it takes to process data depends on the number of images and its resolution. If processing is taking too long, try lowering the resolution of your custom data.
*  **Error: Data processing did not complete:**
    * This means that the data processing script did not fully complete. This could be because there were not enough images, or that the images were of low quality. We recommend images with little to no motion blur and lots of visual overlap of the scene to increase the chances of successful processing.
*   **Training is not showing progress**:
    * The lack of output is a bug in Colab. You can see the training progress from the viewer.
* **Viewer Quality is bad / Low resolution**:
    * This may be because more GPU is being used on training that rendering the viewer. Try pausing training or decreasing training utilization.
* **WARNING: Running pip as the 'root' user...:**:
    * This and other pip warnings or errors can be safely ignored.
* **Other problems?**
    * Feel free to create an issue on our [GitHub repo](https://github.com/nerfstudio-project/nerfstudio).


In [ ]:
#@title # Install Conda (requires runtime restart) { vertical-output: true, display-mode: "form" }

!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...


In [ ]:
!conda --version

conda 23.1.0


In [ ]:
#@markdown <h1>Install Nerfstudio and Dependencies (~8 min)</h1>

%cd /content/
!pip install --upgrade pip
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118

!conda install -c "nvidia/label/cuda-11.8.0" cuda-toolkit

# Installing TinyCuda
!pip install ninja "git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch"

# Installing COLMAP
!conda install -c conda-forge colmap


# Install nerfstudio
!pip install nerfstudio


Streaming output truncated to the last 5000 lines.






cudatoolkit-11.8.0   | 682.8 MB  | :   1% 0.0071393994399886074/1 [00:00<01:09, 70.46s/it]











boost-cpp-1.78.0     | 15.2 MB   | :  37% 0.3668090458581742/1 [00:00<00:00,  1.44s/it]










libllvm16-16.0.1     | 33.4 MB   | :  20% 0.199013461696493/1 [00:00<00:02,  2.89s/it]  











boost-cpp-1.78.0     | 15.2 MB   | :  48% 0.48324000704349357/1 [00:00<00:00,  1.25s/it]












cudatoolkit-11.8.0   | 682.8 MB  | :   1% 0.009885322301522687/1 [00:00<00:58, 58.92s/it] 










libllvm16-16.0.1     | 33.4 MB   | :  24% 0.2429272302398506/1 [00:00<00:02,  2.69s/it]






imath-3.1.9          | 158 KB    | : 100% 1.0/1 [00:00<00:00,  1.18it/s]                






imath-3.1.9          | 158 KB    | : 100% 1.0/1 [00:00<00:00,  1.18it/s]











boost-cpp-1.78.0     | 15.2 MB   | :  60% 0.6037924181822756/1 [00:00<00:00,  1.12s/it] 












cudatoolkit-11.8.0   | 682.8 MB  | :   1% 0.012265122114852223/1 [0

In [ ]:
#@markdown <h1> Downloading and Processing Data</h1>
#@markdown <h3>Pick the preset scene or upload your own images/video</h3>
import os
import glob
from google.colab import files
from IPython.core.display import display, HTML

scene = '\uD83D\uDCE4 upload your images' #@param ['🖼 poster', '🚜 dozer', '🌄 desolation', '📤 upload your images' , '🎥 upload your own video', '🔺 upload Polycam data', '💽 upload your own Record3D data']
scene = ' '.join(scene.split(' ')[1:])

if scene == "upload Polycam data":
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    if len(uploaded.keys()) > 1:
        print("ERROR, upload a single .zip file when processing Polycam data")
    dataset_dir = [os.path.join(dir, f) for f in uploaded.keys()][0]
    !ns-process-data polycam --data $dataset_dir --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene == 'upload your own Record3D data':
    display(HTML('<h3>Zip your Record3D folder, and upload.</h3>'))
    display(HTML('<h3>More information on Record3D can be found <a href="https://docs.nerf.studio/en/latest/quickstart/custom_dataset.html#record3d-capture" target="_blank">here</a>.</h3>'))
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    record_3d_zipfile = preupload_datasets[0]
    !unzip $record_3d_zipfile -d /content/data/nerfstudio/custom_data
    custom_data_directory = glob.glob('/content/data/nerfstudio/custom_data/*')[0]
    !ns-process-data record3d --data $custom_data_directory --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene in ['upload your images', 'upload your own video']:
    display(HTML('<h3>Select your custom data</h3>'))
    display(HTML('<p/>You can select multiple images by pressing ctrl, cmd or shift and click.<p>'))
    display(HTML('<p/>Note: This may take time, especially on higher resolution inputs, so we recommend to download dataset after creation.<p>'))
    !mkdir -p /content/data/nerfstudio/custom_data
    if scene == 'upload your images':
        !mkdir -p /content/data/nerfstudio/custom_data/raw_images
        %cd /content/data/nerfstudio/custom_data/raw_images
        uploaded = files.upload()
        dir = os.getcwd()
    else:
        %cd /content/data/nerfstudio/custom_data/
        uploaded = files.upload()
        dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    del uploaded
    %cd /content/

    if scene == 'upload your images':
        !ns-process-data images --data /content/data/nerfstudio/custom_data/raw_images --output-dir /content/data/nerfstudio/custom_data/
    else:
        video_path = preupload_datasets[0]
        !ns-process-data video --data $video_path --output-dir /content/data/nerfstudio/custom_data/

    scene = "custom_data"
else:
    %cd /content/
    !ns-download-data nerfstudio --capture-name=$scene

print("Data Processing Succeeded!")

/content/data/nerfstudio/custom_data/raw_images


Saving 0.jpg to 0.jpg
Saving 1.jpg to 1.jpg
Saving 2.jpg to 2.jpg
Saving 3.jpg to 3.jpg
Saving 4.jpg to 4.jpg
Saving 5.jpg to 5.jpg
Saving 6.jpg to 6.jpg
Saving 7.jpg to 7.jpg
Saving 8.jpg to 8.jpg
Saving 9.jpg to 9.jpg
Saving 10.jpg to 10.jpg
Saving 11.jpg to 11.jpg
Saving 12.jpg to 12.jpg
Saving 13.jpg to 13.jpg
Saving 14.jpg to 14.jpg
Saving 15.jpg to 15.jpg
Saving 16.jpg to 16.jpg
Saving 17.jpg to 17.jpg
Saving 18.jpg to 18.jpg
Saving 19.jpg to 19.jpg
Saving 20.jpg to 20.jpg
Saving 21.jpg to 21.jpg
Saving 22.jpg to 22.jpg
Saving 23.jpg to 23.jpg
/content
[16:38:54] 🎉 Done copying images with prefix 'frame_'.                                        ]8;id=165555;file:///usr/local/lib/python3.10/site-packages/nerfstudio/process_data/process_data_utils.py\process_data_utils.py]8;;\:]8;id=128613;file:///usr/local/lib/python3.10/site-packages/nerfstudio/process_data/process_data_utils.py#329\329]8;;\
(   ●  ) Copying images...
🌔  Running COLMAP feature extractor...
[16:38:57] 🎉 Do

In [ ]:
#@markdown <h1>Set up and Start Viewer</h1>

%cd /content

# Install localtunnel
# We are using localtunnel https://github.com/localtunnel/localtunnel but ngrok could also be used
!npm install -g localtunnel

# Tunnel port 7007, the default for
!rm url.txt 2> /dev/null
get_ipython().system_raw('lt --port 7007 >> url.txt 2>&1 &')

import time
time.sleep(3) # the previous command needs time to write to url.txt


with open('url.txt') as f:
  lines = f.readlines()
websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
# from nerfstudio.utils.io import load_from_json
# from pathlib import Path
# json_filename = "nerfstudio/nerfstudio/viewer/app/package.json"
# version = load_from_json(Path(json_filename))["version"]
url = f"https://viewer.nerf.studio/?websocket_url={websocket_url}"
print(url)
print("You may need to click Refresh Page after you start training!")
# from IPython import display
# display.IFrame(src=url, height=800, width="100%")

/content
/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
updated 1 package in 0.591s
https://viewer.nerf.studio/?websocket_url=wss://fancy-rats-marry.loca.lt
You may need to click Refresh Page after you start training!


In [ ]:
#@markdown <h1>Start Training</h1>

%cd /content
if os.path.exists(f"data/nerfstudio/{scene}/transforms.json"):
    !ns-train tensorf --viewer.websocket-port 7007 nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4
else:
    from IPython.core.display import display, HTML
    display(HTML('<h3 style="color:red">Error: Data processing did not complete</h3>'))
    display(HTML('<h3>Please re-run `Downloading and Processing Data`, or view the FAQ for more info.</h3>'))

Streaming output truncated to the last 5000 lines.
----------------------------------------------------------------------------------------------------     
Viewer at: https://viewer.nerf.studio/versions/23-05-15-1/?websocket_url=ws://localhost:7007             
Step (% Done)       Vis Rays / Sec       Train Iter (time)    ETA (time)           Train Rays / Sec      
-------------------------------------------------------------------------------------------------------- 
22390 (74.63%)      231.496 ms           29 m, 21 s           17.69 K                                    
22400 (74.67%)      231.792 ms           29 m, 21 s           17.67 K                                    
22410 (74.70%)      232.494 ms           29 m, 24 s           17.62 K                                    
22420 (74.73%)      236.308 ms           29 m, 51 s           17.34 K                                    
22430 (74.77%)      241.287 ms           30 m, 26 s           16.98 K                                

In [ ]:
!ns-export poisson --load-config outputs/unnamed/nerfacto/2023-10-09_132739/config.yml --output-dir exports/mesh/ --target-num-faces 50000 --num-pixels-per-side 2048 --normal-method open3d --num-points 1000000 --remove-outliers True --use-bounding-box True --bounding-box-min -1 -1 -1 --bounding-box-max 1 1 1

2023-10-09 15:07:12.552711: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Setting up training dataset...
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Caching all 324 images.
Loading data batch ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:02
Setting up evaluation dataset...
Caching all 35 images.
Loading data batch ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Loading latest checkpoint from load_dir
✅ Done loading checkpoint from outputs/unnamed/nerfacto/2023-10-09_132739/nerfstudio_models/step-0000299

In [ ]:
!ns-export pointcloud --load-config outputs/unnamed/nerfacto/2023-10-09_132739/config.yml --output-dir exports/pcd/ --num-points 1000000 --remove-outliers True --normal-method open3d --use-bounding-box True --bounding-box-min -1 -1 -1 --bounding-box-max 1 1 1

2023-10-09 15:13:59.679123: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Setting up training dataset...
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Caching all 324 images.
Loading data batch ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:02
Setting up evaluation dataset...
Caching all 35 images.
Loading data batch ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Loading latest checkpoint from load_dir
✅ Done loading checkpoint from outputs/unnamed/nerfacto/2023-10-09_132739/nerfstudio_models/step-0000299

In [ ]:
#@title # Render Video { vertical-output: true }
#@markdown <h3>Export the camera path from within the viewer, then run this cell.</h3>
#@markdown <h5>The rendered video should be at renders/output.mp4!</h5>


base_dir = "/content/outputs/unnamed/nerfacto/"
training_run_dir = base_dir + os.listdir(base_dir)[0]

from IPython.core.display import display, HTML
display(HTML('<h3>Upload the camera path JSON.</h3>'))
%cd $training_run_dir
uploaded = files.upload()
uploaded_camera_path_filename = list(uploaded.keys())[0]

config_filename = training_run_dir + "/config.yml"
camera_path_filename = training_run_dir + "/" + uploaded_camera_path_filename
camera_path_filename = camera_path_filename.replace(" ", "\\ ").replace("(", "\\(").replace(")", "\\)")

%cd /content/
!ns-render camera-path --load-config $config_filename --camera-path-filename $camera_path_filename --output-path renders/output.mp4

/content/outputs/unnamed/nerfacto/2023-10-09_132739


In [ ]:
!pwd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp -r /content/outputs /content/drive/MyDrive/AI-Capstone/data_sample/face/nerfstudio_result/

In [ ]:
!cp -r /content/exports /content/drive/MyDrive/AI-Capstone/data_sample/face/nerfstudio_result/

In [ ]:
!cp -r /content/renders /content/drive/MyDrive/AI-Capstone/data_sample/face/nerfstudio_result/